<a href="https://colab.research.google.com/github/kim-jason/calib-challenge/blob/main/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Test")

Test


In [12]:
print("Test")

Test


In [ ]:
print("Hello world")